<a href="https://colab.research.google.com/github/AdarshXNepal/ChandreTheAssistant/blob/main/Langchain_multimodel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

ALL INSTALLATOIN


In [ ]:
!pip install 'unstructured[all-docs]' pillow lxml
!pip install chromadb
!pip install langchain
!pip install langchain_community
!pip install langchain_google_genai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.3/19.3 MB 90.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 20.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 67.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 88.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.7/55.7 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.2/196.2 kB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.5/118.5 kB 9.4 MB/s eta 0:

In [ ]:
GEMINI_API_KEY=""

In [ ]:
from unstructured.partition.pdf import partition_pdf
output_path='/'
filepath='/multimodal_rag_sample.pdf'




In [ ]:
!apt-get update
!apt-get install -y poppler-utils

In [ ]:
chunks=partition_pdf(
filename=filepath,
                      infer_table_structure=True,
                      strategy="hi_res",
                      extract_image_block_types=['Image'],
                      extract_image_block_to_payload=True,
                      chunking_strategy="by_title",
                      max_characters=1000,
                      combine_text_under_n_chars=500,
                      new_after_n_chars=6000,
)

In [ ]:
set([str(type(el) for el in chunks)])

In [ ]:
chunks[0].metadata.orig_elements

In [ ]:
tables=[]
texts=[]

for chunk in chunks:
  if 'Table' in str(type(chunk)):
    tables.append(chunk)
  else:
    texts.append(chunk)



In [ ]:
def get_image_base64(chunks):
  image_b64=[]
  for chunk in chunks:
    if "CompositeElement" in str(type(chunk)):
      chunk_el=chunk.metadata.orig_elements
      for el in chunk_el:
        if "Image" in str(type(el)):
          image_b64.append(el.metadata.image_base64)
  return image_b64

images=get_image_base64(chunks)



In [ ]:
!apt-get install tesseract-ocr libmagic-dev

In [ ]:
import base64
from IPython.display import Image, display

def display_base64_img(base64_code):
  #decode base64 string to binary
  image_data=base64.b64decode(base64_code)
  #display image
  display(Image(data=image_data))

display_base64_img(images[2])

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

prompt_text="""
You are an assistant tasked with summarizing tables and text.
give a concise summary of the table or text

respond only wuth the summary,no additoinal comment.
Dono start your own message by saying "Here is a summary" or anything like that.
Just give sumamrt as it is

Table or text chunk :{element}
"""

prompt=ChatPromptTemplate.from_template(prompt_text)

model=ChatGoogleGenerativeAI(model="gemini-2.0-flash", google_api_key="")

In [ ]:
summarize_chain={"element":lambda x:x} | prompt | model | StrOutputParser()

In [ ]:
#summarize text

text_summaries=summarize_chain.batch(texts,{"max_concurrency":3})

#summarze tables

tables_html=[table.metadata.text_as_html for table in tables]
table_summaries=summarize_chain.batch(tables_html,{"max_concurrency":3})


In [ ]:
text_summaries

In [ ]:
prompt_template="""
Describe the image in detail.
Be specific of the content present in the image, graphs , bar plots and other
"""

messages=[
    (
        "user",
        [{
            "type":"text","text":prompt_template
        },
         {
             "type":"image_url",
             "image_url":{"url":"data:image/jpeg;base64,{image}"},
         },

         ],

    )
]

prompt =ChatPromptTemplate.from_messages(messages)

chain=prompt | model | StrOutputParser()

image_summaries=chain.batch(images)

In [ ]:
image_summaries

In [ ]:
import uuid
from langchain.vectorstores import Chroma
from langchain.storage import InMemoryStore
from langchain.schema.document import Document
# Remove the direct import of genai, as GoogleGenerativeAIEmbeddings handles the client
# from google import genai
from langchain.retrievers.multi_vector import MultiVectorRetriever
from langchain_google_genai import GoogleGenerativeAIEmbeddings # Import the correct embedding class


# Replace the custom function with the Langchain embedding class
# def gemini_embedding_function(texts):
#   client = genai.Client(api_key="")

#   result = client.models.embed_content(
#         model="gemini-embedding-exp-03-07",
#         content=texts);
#   return result.embeddings

# Initialize the GoogleGenerativeAIEmbeddings with the API key and model
gemini_embedding_function = GoogleGenerativeAIEmbeddings(
    model="models/embedding-001", # Use a supported embedding model name
    google_api_key=""
)


vectorstore=Chroma(collection_name="multi_model_rag", embedding_function=gemini_embedding_function)

store=InMemoryStore(
)
id_key="doc_id"

#the retreiver (empty to start)
retriever=MultiVectorRetriever(
    vectorstore=vectorstore,
    docstore=store,
    id_key=id_key,

)

In [ ]:
#load and find

doc_ids=[str(uuid.uuid4()) for _ in texts]
summary_texts=[
    Document(page_content=summary,metadata={id_key:doc_ids[i]}) for i,summary in enumerate(text_summaries)

]
retriever.vectorstore.add_documents(summary_texts)
retriever.docstore.mset(list(zip(doc_ids,texts)))


#addTables

table_ids=[str(uuid.uuid4() for _ in tables)]
summary_tables=[
    Document(page_content=summary,metadata={id_key:table_ids[i]}) for i,summary in enumerate(table_summaries)

]
if summary_tables:
  retriever.vectorstore.add_documents(summary_tables)
  retriever.docstore.mset(list(zip(table_ids,tables)))

#addimages
image_ids=[str(uuid.uuid4()) for _ in images]
summary_images=[
    Document(page_content=summary,metadata={id_key:image_ids[i]}) for i,summary in enumerate(image_summaries)

]
retriever.vectorstore.add_documents(summary_images)
retriever.docstore.mset(list(zip(image_ids,images)))


In [ ]:
docs=retriever.invoke("what is in the chroma")

In [ ]:
for doc in docs:
  print(str(doc)+"\n\n"+"-"*80)

In [ ]:
from langchain_core.runnables import RunnablePassthrough, RunnableLambda
from langchain_core.messages import SystemMessage, HumanMessage
from base64 import b64decode


def parse_docs(docs):
    """Split base64-encoded images and texts"""
    b64 = []
    text = []
    for doc in docs:
        try:
            b64decode(doc)
            b64.append(doc)
        except Exception as e:
            text.append(doc)
    return {"images": b64, "texts": text}


def build_prompt(kwargs):

    docs_by_type = kwargs["context"]
    user_question = kwargs["question"]

    context_text = ""
    if len(docs_by_type["texts"]) > 0:
        for text_element in docs_by_type["texts"]:
            context_text += text_element.text

    # construct prompt with context (including images)
    prompt_template = f"""
    Answer the question based only on the following context, which can include text, tables, and the below image.
    Context: {context_text}
    Question: {user_question}
    """

    prompt_content = [{"type": "text", "text": prompt_template}]

    if len(docs_by_type["images"]) > 0:
        for image in docs_by_type["images"]:
            prompt_content.append(
                {
                    "type": "image_url",
                    "image_url": {"url": f"data:image/jpeg;base64,{image}"},
                }
            )

    return ChatPromptTemplate.from_messages(
        [
            HumanMessage(content=prompt_content),
        ]
    )


chain = (
    {
        "context": retriever | RunnableLambda(parse_docs),
        "question": RunnablePassthrough(),
    }
    | RunnableLambda(build_prompt)
    | model
    | StrOutputParser()
)

chain_with_sources = {
    "context": retriever | RunnableLambda(parse_docs),
    "question": RunnablePassthrough(),
} | RunnablePassthrough().assign(
    response=(
        RunnableLambda(build_prompt)
        | model
        | StrOutputParser()
    )
)

In [ ]:
import base64
from IPython.display import Image, display

def display_base64_img(base64_code):
  #decode base64 string to binary
  image_data=base64.b64decode(base64_code)
  #display image
  display(Image(data=image_data))



In [ ]:
response = chain_with_sources.invoke(
    "What is multimodel rag with image"
)

print("Response:", response['response'])

print("\n\nContext:")
for text in response['context']['texts']:
    print(text.text)
    print("Page number: ", text.metadata.page_number)
    print("\n" + "-"*50 + "\n")
for image in response['context']['images']:
    display_base64_img(image)